In [16]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import add, Dropout, Activation, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras import models
from sklearn.metrics import r2_score
from matplotlib.patches import Patch
from scipy import stats

import warnings
warnings.filterwarnings(action='ignore')

In [17]:
#파일 불러오기
excel_data=pd.read_excel('데이터정리_취합_소.xlsx', sheet_name='ResultWeight_MainDimension', header=1)
excel_data[0:8]

,돼지,D1,rear_W,rear_trunk_W,rear_trunk_A,trunk_W,trunk_H,trunk_A,trunk_G,trunk_front_W,...,max1,mean2,max2,mean3,max3,mean4,max4,mean5,max5,weight
0,상관계수,0.224671,0.490432,0.53374,0.447982,0.844043,0.338332,0.818565,0.747194,0.492227,...,0.820094,0.323015,0.171445,0.562356,0.667816,0.754422,0.781111,0.579229,0.527422,1
1,2022_08_10_164935_760_123454.pcd,1000.010000,626.300000,644.43000,437945.650000,766.350000,881.540000,536281.670000,2618.780000,635.790000,...,782.320000,846.880000,902.330000,0.820000,0.880000,478192.010000,558005.950000,2502.030000,2676.490000,760
2,2022_08_10_164938_760_123454.pcd,1000.010000,613.800000,636.77000,423469.520000,810.680000,858.710000,546742.600000,2641.630000,686.550000,...,814.930000,829.600000,876.770000,0.870000,0.940000,492604.190000,567070.800000,2515.050000,2695.410000,760
3,2022_08_10_164943_760_123454.pcd,1110.010000,571.620000,627.15000,421899.660000,794.610000,843.230000,531064.360000,2600.220000,650.540000,...,802.610000,847.130000,1051.280000,0.820000,0.950000,473963.160000,554214.080000,2508.900000,2869.910000,760
4,2022_08_10_164946_760_123454.pcd,1150.010000,528.660000,565.01000,408052.190000,810.030000,855.260000,549242.080000,2634.540000,653.820000,...,821.240000,811.890000,867.840000,0.870000,0.970000,476599.320000,567133.620000,2476.310000,2680.670000,760
5,2022_08_10_165558_760_123454.pcd,990.010000,608.000000,668.06000,459952.620000,819.000000,852.650000,552299.220000,2655.300000,666.130000,...,833.080000,869.250000,1040.390000,0.830000,0.970000,498735.750000,571712.020000,2567.910000,2760.180000,760
6,2022_08_10_165601_760_123454.pcd,970.010000,622.540000,675.94000,461765.700000,808.440000,853.450000,549666.940000,2649.290000,663.430000,...,813.320000,837.790000,864.830000,0.870000,0.950000,499497.470000,567715.080000,2537.740000,2691.140000,760
7,2022_08_10_165754_700_123456.pcd,890.010000,650.820000,680.61000,483585.780000,801.050000,865.560000,545320.390000,2631.000000,642.490000,...,808.100000,857.220000,886.760000,0.860000,0.940000,517789.490000,567286.700000,2579.130000,2676.750000,700


In [18]:
#무게와 치수와 가지는 상관관계 확인
data=excel_data[1:]
corr=excel_data[0:1]

corr

,돼지,D1,rear_W,rear_trunk_W,rear_trunk_A,trunk_W,trunk_H,trunk_A,trunk_G,trunk_front_W,...,max1,mean2,max2,mean3,max3,mean4,max4,mean5,max5,weight
0,상관계수,0.224671,0.490432,0.53374,0.447982,0.844043,0.338332,0.818565,0.747194,0.492227,...,0.820094,0.323015,0.171445,0.562356,0.667816,0.754422,0.781111,0.579229,0.527422,1


In [19]:

#면적, 너비, 높이, 둘레 관련 데이터끼리 묶기
data_rear=pd.concat([data['rear_W']], axis=1)
data_rear_trunk=pd.concat([data['rear_trunk_W'], data['rear_trunk_W']], axis=1)
data_trunk=pd.concat([data['trunk_W'], data['trunk_H'], data['trunk_A'],data['trunk_G']], axis=1)
data_trunk_front=pd.concat([data['trunk_front_W'], data['trunk_front_H'], data['trunk_front_A'],data['trunk_front_G']], axis=1)
data_front=pd.concat([data['front_W']], axis=1)
data_mean=pd.concat([data['mean1'], data['mean2'], data['mean3'],data['mean4'], data['mean5']], axis=1)
data_max=pd.concat([data['max1'], data['max2'], data['max3'],data['max4'], data['max5']], axis=1)


In [20]:
#모든 변수를 입력변수로 넣기
data_All=pd.concat([data_rear,data_rear_trunk,data_trunk,data_trunk_front,data_front, data_mean, data_max], axis=1)

In [21]:
#data70=pd.concat([data['trunk_W'], data['trunk_A'], data['trunk_G'], 
                  #data['mean1'], data['max1'], data['mean4'], data['max4']], axis=1)

In [22]:
# 입력변수 X(치수), 출력변수 Y(무게)
X=data_All
Y=data['weight']


X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.20, random_state=1)

In [23]:
#데이터 타입 맞추기 (Y값 dataframe 씌우기)
Y_train=pd.DataFrame(Y_train)
Y_test=pd.DataFrame(Y_test)

In [24]:
#MinMaxScaler 사용하여 데이터 정규화, scaler1은 X, scaler2는 Y 정규화에 쓰일 예정
scaler1= MinMaxScaler()
scaler2= MinMaxScaler()

In [25]:
scaler1.fit(X_train)
scaler1.fit(X_test)

MinMaxScaler()

In [26]:
X_train_scaled = scaler1.transform(X_train)
X_test_scaled=scaler1.transform(X_test)

In [27]:
X_scaled = scaler1.fit_transform(X_train)
Y_scaled = scaler2.fit_transform(Y_train)

In [28]:
scaler2.fit(Y_train)
scaler2.fit(Y_test)

MinMaxScaler()

In [29]:
Y_train_scaled = scaler2.transform(Y_train)
Y_test_scaled=scaler2.transform(Y_test)

In [30]:
#모델 설계하기
#4개의 은닉층으로 구성된 Multi-layer-perceptron 모델
#입력층과 은닉층의 노드 갯수는 34개이며, 출력층의 노드는 1개
model = models.Sequential()
model.add(tf.keras.layers.Dense(X_scaled.shape[1], input_dim =X_scaled.shape[1],activation = 'relu'))
model.add(tf.keras.layers.Dense(X.shape[1]/2,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,  activation='sigmoid'))
optimize = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='mse', optimizer=optimize, metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 22)                506       
                                                                 
 dense_4 (Dense)             (None, 11)                253       
                                                                 
 dense_5 (Dense)             (None, 1)                 12        
                                                                 
Total params: 771
Trainable params: 771
Non-trainable params: 0
_________________________________________________________________


In [31]:
#모델 실행
model.fit(X_scaled, Y_scaled, epochs=300)
loss_and_metrics_scaled = model.evaluate(X_test_scaled,Y_test_scaled)
loss_and_metrics = scaler2.inverse_transform([loss_and_metrics_scaled]) 

Epoch 1/300
8/8 [==============================] - 1s 2ms/step - loss: 0.0636 - acc: 0.1084
Epoch 2/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0467 - acc: 0.1044
Epoch 3/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0327 - acc: 0.1205
Epoch 4/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0300 - acc: 0.1205
Epoch 5/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0263 - acc: 0.1205
Epoch 6/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0232 - acc: 0.1205
Epoch 7/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0214 - acc: 0.1205
Epoch 8/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0209 - acc: 0.1205
Epoch 9/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0198 - acc: 0.1205
Epoch 10/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0190 - acc: 0.1205
Epoch 11/300
8/8 [==============================] - 0s 997us/step - loss: 0.019

8/8 [==============================] - 0s 1ms/step - loss: 0.0090 - acc: 0.1205
Epoch 89/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0079 - acc: 0.1205
Epoch 90/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0098 - acc: 0.1205
Epoch 91/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0083 - acc: 0.1205
Epoch 92/300
8/8 [==============================] - 0s 997us/step - loss: 0.0077 - acc: 0.1205
Epoch 93/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0082 - acc: 0.1205
Epoch 94/300
8/8 [==============================] - 0s 997us/step - loss: 0.0099 - acc: 0.1205
Epoch 95/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0080 - acc: 0.1205
Epoch 96/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0085 - acc: 0.1205
Epoch 97/300
8/8 [==============================] - 0s 997us/step - loss: 0.0082 - acc: 0.1205
Epoch 98/300
8/8 [==============================] - 0s 997us/step - loss: 0.0

8/8 [==============================] - 0s 1ms/step - loss: 0.0077 - acc: 0.1205
Epoch 175/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0073 - acc: 0.1205
Epoch 176/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0052 - acc: 0.1205
Epoch 177/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0054 - acc: 0.1205
Epoch 178/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0060 - acc: 0.1205
Epoch 179/300
8/8 [==============================] - 0s 2ms/step - loss: 0.0082 - acc: 0.1205
Epoch 180/300
8/8 [==============================] - 0s 2ms/step - loss: 0.0094 - acc: 0.1205
Epoch 181/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0079 - acc: 0.1205
Epoch 182/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0092 - acc: 0.1205
Epoch 183/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0078 - acc: 0.1205
Epoch 184/300
8/8 [==============================] - 0s 1ms/step - loss: 0

8/8 [==============================] - 0s 1ms/step - loss: 0.0043 - acc: 0.1205
Epoch 262/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0047 - acc: 0.1205
Epoch 263/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0046 - acc: 0.1205
Epoch 264/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0045 - acc: 0.1205
Epoch 265/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0042 - acc: 0.1205
Epoch 266/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0042 - acc: 0.1205
Epoch 267/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0050 - acc: 0.1205
Epoch 268/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0051 - acc: 0.1205
Epoch 269/300
8/8 [==============================] - 0s 997us/step - loss: 0.0047 - acc: 0.1205
Epoch 270/300
8/8 [==============================] - 0s 1ms/step - loss: 0.0046 - acc: 0.1205
Epoch 271/300
8/8 [==============================] - 0s 1ms/step - loss:

In [32]:
#Y 예측값
Y_train_pred_scaled=model.predict(X_train_scaled)
Y_test_pred= model.predict(X_test_scaled)

2/2 [==============================] - 0s 2ms/step


In [33]:
Y_train_pred_scaled #정규화 된 값 역변환 필요

array([[9.97219160e-02],
       [2.90864766e-01],
       [9.02532756e-01],
       [4.86912698e-01],
       [5.82910359e-01],
       [3.85291278e-01],
       [5.66447258e-01],
       [3.13249320e-01],
       [4.36454326e-01],
       [8.66090953e-01],
       [1.90590799e-01],
       [2.93485403e-01],
       [4.80130374e-01],
       [1.49909745e-03],
       [2.63329178e-01],
       [4.64042239e-02],
       [6.56134307e-01],
       [1.86672896e-01],
       [5.20075142e-01],
       [4.85201329e-01],
       [9.23360705e-01],
       [5.01173019e-01],
       [4.76983994e-01],
       [6.59004040e-03],
       [2.10824624e-01],
       [1.30454466e-01],
       [2.95173794e-01],
       [3.55032235e-01],
       [2.46325001e-01],
       [4.34003145e-01],
       [8.76986049e-03],
       [4.85831499e-01],
       [9.20232058e-01],
       [4.81522754e-02],
       [9.00410354e-01],
       [4.90686685e-01],
       [1.57206450e-04],
       [5.23681521e-01],
       [4.56582725e-01],
       [1.27820149e-01],


In [34]:
Y_train_pred = scaler2.inverse_transform(Y_train_pred_scaled)
Y_test_pred = scaler2.inverse_transform(Y_test_pred)

In [35]:
#훈련데이터 오차율
train_error_rate=np.mean(abs(Y_train_pred-Y_train)*100/Y_train)
train_error_rate

weight    3.457379
dtype: float64

In [36]:
#검증데이터 오차율
error_rate=np.mean(abs(Y_test_pred-Y_test)*100/Y_test)
error_rate

weight    4.133673
dtype: float64

In [37]:
#회귀모델의 척도로 결정계수 확인
R2=r2_score(Y_test, Y_test_pred)
R2

0.424840158031234

In [38]:
#RMSE 구하기
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(Y_test, Y_test_pred)**0.5
RMSE

40.81300588304251

In [39]:
#결과 값 엑셀 파일로 확인하기
df_Y_test=pd.DataFrame(Y_test)
df_Y_test_pred=pd.DataFrame(Y_test_pred)
df_Y_train=pd.DataFrame(Y_train)
df_Y_train_pred=pd.DataFrame(Y_train_pred)

In [40]:
pd.set_option('display.max_rows',200)
df_Y_test.to_excel('55.xlsx')

In [41]:
pd.set_option('display.max_rows',200)
df_Y_test_pred.to_excel('66.xlsx')

In [42]:
pd.set_option('display.max_rows',200)
df_Y_train.to_excel('77.xlsx')

In [43]:
pd.set_option('display.max_rows',200)
df_Y_train_pred.to_excel('88.xlsx')